# Automatización con Python

In [ ]:
import snowflake.connector
import os
import configparser


config_path = os.path.join(os.environ['USERPROFILE'], '.snowsql', 'config')
config = configparser.ConfigParser()
config.read(config_path)

try:
    account = config['connections.example']['accountname']
    user = config['connections.example']['username']
    password = config['connections.example']['password']
except KeyError as e:
    print(f"Error: No se encontraron las credenciales en el archivo de configuración: {e}")
    exit()

try:
    conn = snowflake.connector.connect(
        user=user,
        password=password,
        account=account
    )
    print("Conexión exitosa a Snowflake")

except Exception as e:
    print(f"Error de conexión: {e}")

In [ ]:
# Variables de configuración
WAREHOUSE = 'COMPUTE_WH'
DATABASE = 'DATABUSTER_SAMPLE'
SCHEMA = 'PUBLIC'
TABLE = 'CAMPAIGN_DATA_PYTHON'
path_archivo = os.path.join(os.getcwd(), 'data', 'dataset.csv').replace('\\', '/')

c:/Users/flavi/Teaching/curso-udemy-databoosters-snowflake-analisisdatos-basico/15-Conexion con Python/data/dataset.csv


In [ ]:
conn.cursor().execute(f"USE WAREHOUSE {WAREHOUSE}")
conn.cursor().execute(f"USE DATABASE {DATABASE}")  
conn.cursor().execute(f"USE SCHEMA {SCHEMA}")

In [ ]:
sql_ddl = f"""
CREATE OR REPLACE TABLE {DATABASE}.{SCHEMA}.{TABLE} (
    DateStart DATE,
    Campaign STRING,
    Region STRING,
    Clicks NUMBER,
    Impressions NUMBER,
    Views NUMBER,
    Cost NUMBER
);
"""
conn.cursor().execute(sql_ddl)

In [ ]:
conn.cursor().execute(f"PUT file://{path_archivo} @%{TABLE}") # Carga el archivo CSV al stage de la tabla
conn.cursor().execute(f"""
COPY INTO {TABLE} 
FILE_FORMAT=(
    TYPE=CSV SKIP_HEADER=1)
""")

In [ ]:
import pandas as pd
sql = f"""
SELECT * FROM {TABLE}
"""
cursor = conn.cursor().execute(sql)
df = cursor.fetch_pandas_all()
print(df)